In [ ]:
#Importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.figure_factory as ff
from tensorflow import keras

In [ ]:
stock_price = pd.read_csv('stock.csv')
stock_price

,Date,AAPL,BA,T,MGM,AMZN,IBM,TSLA,GOOG,sp500
0,2012-01-12,60.198570,75.510002,30.120001,12.130000,175.929993,180.550003,28.250000,313.644379,1295.500000
1,2012-01-13,59.972858,74.599998,30.070000,12.350000,178.419998,179.160004,22.790001,311.328064,1289.089966
2,2012-01-17,60.671429,75.239998,30.250000,12.250000,181.660004,180.000000,26.600000,313.116364,1293.670044
3,2012-01-18,61.301430,75.059998,30.330000,12.730000,189.440002,181.070007,26.809999,315.273285,1308.040039
4,2012-01-19,61.107143,75.559998,30.420000,12.800000,194.449997,180.520004,26.760000,318.590851,1314.500000
...,...,...,...,...,...,...,...,...,...,...
2154,2020-08-05,440.250000,174.279999,29.850000,16.719999,3205.030029,125.449997,1485.020020,1473.609985,3327.770020
2155,2020-08-06,455.609985,172.199997,29.840000,18.459999,3225.000000,126.120003,1489.579956,1500.099976,3349.159912
2156,2020-08-07,444.450012,170.020004,30.020000,19.030001,3167.459961,124.959999,1452.709961,1494.489990,3351.280029
2157,2020-08-10,450.910004,179.410004,30.200001,21.650000,3148.159912,127.110001,1418.569946,1496.099976,3360.469971


In [ ]:
stock_volume = pd.read_csv('stock_volume.csv')
stock_volume

,Date,AAPL,BA,T,MGM,AMZN,IBM,TSLA,GOOG,sp500
0,2012-01-12,53146800,3934500,26511100,17891100,5385800,6881000,729300,3764400,4019890000
1,2012-01-13,56505400,4641100,22096800,16621800,4753500,5279200,5500400,4631800,3692370000
2,2012-01-17,60724300,3700100,23500200,15480800,5644500,6003400,4651600,3832800,4010490000
3,2012-01-18,69197800,4189500,22015000,18387600,7473500,4600600,1260200,5544000,4096160000
4,2012-01-19,65434600,5397300,25524000,14022900,7096000,8567200,1246300,12657800,4465890000
...,...,...,...,...,...,...,...,...,...,...
2154,2020-08-05,30498000,46551000,22991700,18914200,3930000,3675400,4978000,1979500,4732220000
2155,2020-08-06,50607200,32921600,21908700,35867700,3940600,3417100,5992300,1995400,4267490000
2156,2020-08-07,49453300,19301600,30398500,34530300,3929600,3651000,8883500,1576600,4104860000
2157,2020-08-10,53100900,35857700,35514400,71219700,3167300,3968300,7522300,1289300,4318570000


In [ ]:
#Shorting Dataset according to  Date
stock_price = stock_price.sort_values(by = ['Date'])
stock_volume = stock_volume.sort_values(by = ['Date'])

In [ ]:
stock_price.isnull().sum()

Date     0
AAPL     0
BA       0
T        0
MGM      0
AMZN     0
IBM      0
TSLA     0
GOOG     0
sp500    0
dtype: int64

In [ ]:
stock_volume.isnull().sum()
stock_price.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2159 entries, 0 to 2158
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    2159 non-null   object 
 1   AAPL    2159 non-null   float64
 2   BA      2159 non-null   float64
 3   T       2159 non-null   float64
 4   MGM     2159 non-null   float64
 5   AMZN    2159 non-null   float64
 6   IBM     2159 non-null   float64
 7   TSLA    2159 non-null   float64
 8   GOOG    2159 non-null   float64
 9   sp500   2159 non-null   float64
dtypes: float64(9), object(1)
memory usage: 185.5+ KB


In [ ]:
stock_price.describe()
stock_volume.describe()

,AAPL,BA,T,MGM,AMZN,IBM,TSLA,GOOG,sp500
count,2.159000e+03,2.159000e+03,2.159000e+03,2.159000e+03,2.159000e+03,2.159000e+03,2.159000e+03,2.159000e+03,2.159000e+03
mean,5.820332e+07,6.419916e+06,2.832131e+07,9.845582e+06,4.102673e+06,4.453090e+06,7.001302e+06,2.498238e+06,3.680732e+09
std,4.568141e+07,9.711873e+06,1.428911e+07,7.295753e+06,2.290722e+06,2.462811e+06,5.781208e+06,1.928407e+06,8.622717e+08
min,1.136200e+07,7.889000e+05,6.862400e+06,9.507000e+05,8.813000e+05,1.193000e+06,3.649000e+05,7.900000e+03,1.248960e+09
25%,2.769930e+07,3.031850e+06,2.002150e+07,5.796450e+06,2.675700e+06,3.111250e+06,3.433450e+06,1.325400e+06,3.211890e+09
50%,4.209420e+07,3.991000e+06,2.485930e+07,7.899800e+06,3.494800e+06,3.825000e+06,5.581100e+06,1.813900e+06,3.526890e+09
75%,7.182480e+07,5.325900e+06,3.210565e+07,1.104055e+07,4.768150e+06,4.937300e+06,8.619550e+06,3.245350e+06,3.933290e+09
max,3.765300e+08,1.032128e+08,1.950827e+08,9.009820e+07,2.385610e+07,3.049020e+07,6.093880e+07,2.497790e+07,9.044690e+09


In [ ]:
#Function to normalize stock price based on initial price
def normalize(df):
  x = df.copy()
  for i in x.columns[1:]:
    x[i] = x[i]/x[i][0]
  return x

In [ ]:
#Function to plot interactive plot using plotpy
def interactive_plot(df, title):
  fig = px.line(title = title)
  for i in df.columns[1:]:
    fig.add_scatter(x = df['Date'], y = df[i], name = i)
  fig.show()

In [ ]:
interactive_plot(stock_price , 'Stock Price')

In [ ]:
interactive_plot(stock_volume, "Stock Volume")

In [ ]:
interactive_plot(normalize(stock_price), "Normalized Stock Price")

In [ ]:
#Concatinting Date, Price and Volume in Stock data
def individual_data(price, volume, name):
  return pd.DataFrame({'Date':price['Date'], 'Close':price[name], 'Volume': volume[name]})

In [ ]:
#Function to return the input/output data
def trading_window(data):
  n = 1
  data['Target'] = data[['Close']].shift(-n)
  return data

In [ ]:
# Let's test the functions and get individual stock prices and volumes for AAPL
price_volume_df = individual_data(stock_price, stock_volume, 'sp500')
price_volume_df

,Date,Close,Volume
0,2012-01-12,1295.500000,4019890000
1,2012-01-13,1289.089966,3692370000
2,2012-01-17,1293.670044,4010490000
3,2012-01-18,1308.040039,4096160000
4,2012-01-19,1314.500000,4465890000
...,...,...,...
2154,2020-08-05,3327.770020,4732220000
2155,2020-08-06,3349.159912,4267490000
2156,2020-08-07,3351.280029,4104860000
2157,2020-08-10,3360.469971,4318570000


In [ ]:
# Get the close and volume data as training data (Input)
training_data = price_volume_df.iloc[:, 1:3].values
training_data

array([[1.29550000e+03, 4.01989000e+09],
       [1.28908997e+03, 3.69237000e+09],
       [1.29367004e+03, 4.01049000e+09],
       ...,
       [3.35128003e+03, 4.10486000e+09],
       [3.36046997e+03, 4.31857000e+09],
       [3.33368994e+03, 5.08765000e+09]])

In [ ]:
# Normalize the data
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_data)

In [ ]:
# Create the training and testing data, training data contains present day and previous day values
X = []
y = []
for i in range(1, len(price_volume_df)):
    X.append(training_set_scaled [i-1:i, 0])
    y.append(training_set_scaled [i, 0])

In [ ]:
X

array([[0.00828228],
       [0.00524163],
       [0.00741423],
       ...,
       [0.98245348],
       [0.98345918],
       [0.9878185 ]])

In [ ]:
X = np.asarray(X)
y = np.asarray(y)

In [ ]:
# Split the data
split = int(0.7 * len(X))
X_train = X[:split]
y_train = y[:split]
X_test = X[split:]
y_test = y[split:]

In [ ]:
#Reshaping 1D array to 3D array
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test,(X_test.shape[0], X_test.shape[1], 1))
X_train.shape,X_test.shape

((1510, 1, 1), (648, 1, 1))

In [ ]:
# Create the model
inputs = keras.layers.Input(shape=(X_train.shape[1], X_train.shape[2]))
x = keras.layers.LSTM(150, return_sequences= True)(inputs)
x = keras.layers.Dropout(0.3)(x)
x = keras.layers.LSTM(150, return_sequences=True)(x)
x = keras.layers.Dropout(0.3)(x)
x = keras.layers.LSTM(150)(x)
outputs = keras.layers.Dense(1, activation='linear')(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss="mse")
model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 1, 1)]            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 1, 150)            91200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 1, 150)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 1, 150)            180600    
_________________________________________________________________
dropout_3 (Dropout)          (None, 1, 150)            0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 150)               180600    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                

In [ ]:
#Train model
history = model.fit(X_train, y_train, epochs = 20 , batch_size = 32, validation_split= 0.2)

Epoch 1/20
38/38 [==============================] - 1s 27ms/step - loss: 0.0333 - val_loss: 0.0694
Epoch 2/20
38/38 [==============================] - 0s 6ms/step - loss: 0.0080 - val_loss: 0.0071
Epoch 3/20
38/38 [==============================] - 0s 7ms/step - loss: 8.2117e-04 - val_loss: 5.8917e-04
Epoch 4/20
38/38 [==============================] - 0s 6ms/step - loss: 4.1309e-04 - val_loss: 2.1974e-04
Epoch 5/20
38/38 [==============================] - 0s 6ms/step - loss: 3.3022e-04 - val_loss: 3.5447e-04
Epoch 6/20
38/38 [==============================] - 0s 6ms/step - loss: 3.4344e-04 - val_loss: 1.9972e-04
Epoch 7/20
38/38 [==============================] - 0s 7ms/step - loss: 3.3202e-04 - val_loss: 3.2985e-04
Epoch 8/20
38/38 [==============================] - 0s 7ms/step - loss: 2.9756e-04 - val_loss: 1.0701e-04
Epoch 9/20
38/38 [==============================] - 0s 6ms/step - loss: 3.0442e-04 - val_loss: 3.4815e-05
Epoch 10/20
38/38 [==============================] - 0s 6ms/s

In [ ]:
predicted = model.predict(X)

In [ ]:
predicted

array([[0.0096162 ],
       [0.00656289],
       [0.00874455],
       ...,
       [0.93948287],
       [0.9403536 ],
       [0.94412494]], dtype=float32)

In [ ]:
# Append the predicted values to the list
test_predicted = []

for i in predicted:
  test_predicted.append(i[0])
  #test_predicted.append(i[0][0])

In [ ]:
df_predicted = price_volume_df[1:][['Date']]
df_predicted

,Date
1,2012-01-13
2,2012-01-17
3,2012-01-18
4,2012-01-19
5,2012-01-20
...,...
2154,2020-08-05
2155,2020-08-06
2156,2020-08-07
2157,2020-08-10


In [ ]:
df_predicted['Prediction'] = test_predicted
df_predicted

,Date,Prediction
1,2012-01-13,0.009616
2,2012-01-17,0.006563
3,2012-01-18,0.008745
4,2012-01-19,0.015589
5,2012-01-20,0.018666
...,...,...
2154,2020-08-05,0.921913
2155,2020-08-06,0.930684
2156,2020-08-07,0.939483
2157,2020-08-10,0.940354


In [ ]:
close = []
for i in training_set_scaled:
  close.append(i[0])

close

[0.008282282297732424,
 0.0052416276750753,
 0.00741422696906191,
 0.0142307573843935,
 0.01729509530784823,
 0.017712533229583327,
 0.018006633177067966,
 0.017366260479376172,
 0.02277870847379071,
 0.019159350140567022,
 0.018163150636518743,
 0.01658830576800918,
 0.016303702004927256,
 0.02184417795686766,
 0.02253203252528968,
 0.031715607508639554,
 0.03144519086195274,
 0.03273549031348555,
 0.03411583203621682,
 0.03505980086579574,
 0.030643552849788036,
 0.03497444905223135,
 0.03437200416912045,
 0.030923407809130832,
 0.037948686358382555,
 0.039461862239767,
 0.039926724634844146,
 0.037768427726387443,
 0.04051967285919389,
 0.04160122417680656,
 0.04247877616423823,
 0.04465612378760542,
 0.04157279302098682,
 0.04556210693085683,
 0.0434464861663616,
 0.04093236245154841,
 0.030985076796256172,
 0.03538239031520529,
 0.04168188600709555,
 0.04403468606133076,
 0.044139031192417466,
 0.05593157836290619,
 0.05513943653514419,
 0.05908607477541117,
 0.05983085000157784,


In [ ]:
df_predicted['Close'] = close[1:]
df_predicted

,Date,Prediction,Close
1,2012-01-13,0.009616,0.005242
2,2012-01-17,0.006563,0.007414
3,2012-01-18,0.008745,0.014231
4,2012-01-19,0.015589,0.017295
5,2012-01-20,0.018666,0.017713
...,...,...,...
2154,2020-08-05,0.921913,0.972307
2155,2020-08-06,0.930684,0.982453
2156,2020-08-07,0.939483,0.983459
2157,2020-08-10,0.940354,0.987819


In [ ]:
# Plot the data
interactive_plot(df_predicted, "Original Vs Prediction")